In [1]:
import plotly.io as pio
pio.renderers.default = "browser"

In [2]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils
import instrument

In [3]:
pair = ''
granularity = ''
ma_list = [8, 12]
i_pair = ''

In [7]:
df = pd.read_csv('data/his_data_BTC_USD_1m.csv')
df_plot = df.iloc[0:2500].copy()

In [8]:

#df_ma = df_plot[['open_time','close','volume']].copy()
df_ma = df_plot[['open_time','close']].copy()

In [9]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.close.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)


In [10]:
df_ma['DIFF'] = df_ma.MA_8 - df_ma.MA_12 #calculate a difference between two moving averages when "creating a cross"
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1) #gives previous row value


In [ ]:
def is_trade(row): #function to identify crosses/trades
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 # Golden Cross
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 # Death Cross
    return 0

In [ ]:
#Make new column, apply 
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) #axis means apply fuction to each row
#create dataframe with made trades
df_trades = df_ma[df_ma.IS_TRADE != 0].copy()
df_ma.reset_index(drop=True, inplace=True)
df_trades.shape
#df_trades.head()

,open_time,close,MA_8,MA_12,DIFF,DIFF_PREV,IS_TRADE
3,2025-11-01 00:14:00+00:00,109471.44,109454.25250,109449.895833,4.356667,-0.369583,1
14,2025-11-01 00:25:00+00:00,109519.99,109479.51000,109479.613333,-0.103333,3.364167,-1
16,2025-11-01 00:27:00+00:00,109531.94,109492.92375,109492.157500,0.766250,-1.109167,1
23,2025-11-01 00:34:00+00:00,109445.34,109480.96750,109491.249167,-10.281667,4.573333,-1
30,2025-11-01 00:41:00+00:00,109492.00,109481.82125,109472.561667,9.259583,-1.875417,1


In [13]:
fig = go.Figure()

fig.add_trace(go.Candlestick(   #Adding candlestick chart
    x=df_plot['open_time'], #candlesticks
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    #volume=df_plot['volume'],
    line=dict(width=1), opacity=1, #coloring
    increasing_line_color='#2EC866', 
    decreasing_line_color='#FF3A4C',
    increasing_fillcolor='#24A06B', 
    decreasing_fillcolor='#CC2E3B',
    )) 

for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(    #Adding Moving Average line
        x=df_plot.open_time,
        y=df_ma[col],
        line=dict(width=2),
        line_shape='spline', #makes smooth curve
        name=col
        ))

fig.update_layout(width=1100, height=450,
                  margin=dict(l=30, r=30, t=30, b=30),
                  font=dict(color='#e1e1e1'), 
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e')

fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
 
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()

In [14]:
#calculate difference between trade prices, shifted by 1 to avoid lookahead bias
#df_trades['DELTA'] = (df_trades.close.diff()/i_pair.pipLocation).shift(-1) 
df_trades['DELTA'] = df_trades.close.diff().shift(-1)
df_trades['GAINS'] = df_trades["DELTA"] * df_trades["IS_TRADE"] #calculate gains based on trade direction


In [23]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236 entries, 3 to 2481
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   open_time  236 non-null    datetime64[ns, tzutc()]
 1   close      236 non-null    float64                
 2   MA_8       236 non-null    float64                
 3   MA_12      236 non-null    float64                
 4   DIFF       236 non-null    float64                
 5   DIFF_PREV  236 non-null    float64                
 6   IS_TRADE   236 non-null    int64                  
 7   DELTA      235 non-null    float64                
 8   GAINS      235 non-null    float64                
 9   DURATION   235 non-null    timedelta64[ns]        
dtypes: datetime64[ns, tzutc()](1), float64(7), int64(1), timedelta64[ns](1)
memory usage: 20.3 KB


In [ ]:
from dateutil.parser import *
df_trades['open_time'] = [parse(x) for x in df_trades['open_time']]
df_trades['DURATION'] = df_trades['open_time'].diff().shift(-1)
df_trades['DURATION'] = [x.total_seconds() / 3600 for x in df_trades.DURATION]